In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"w
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#train = pd.read_csv('/kaggle/input/recsysmasterfds-2024/train.csv')
complete_train = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 6/Recommenders/Assignments/Assignment 1/data/train.csv')
complete_train_og = complete_train.copy()

In [ ]:
#from sklearn.model_selection import train_test_split
#small_train, small_val = train_test_split(complete_train, test_size=0.2, random_state=42)

In [ ]:
#filtered_df = traindata[traindata['user_id'] <= 100]
#filtered_df.to_csv('/content/drive/My Drive/_Universität_HPI/Semester 6/Recommenders/Assignments/Assignment 1/Attempts/smalldata.csv', index=False)

In [ ]:
#filtered_df.shape
#traindata= filtered_df

In [ ]:
test = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 6/Recommenders/Assignments/Assignment 1/givenExample/kaggle_baseline.csv')

# Data Analysis and Preprocessing


## Analysis and Overview

In [ ]:
complete_train


,user_id,title,movie_id,rating,release_date,sex,age
0,2592,Top Gun (1986),1101,4,Action|Romance,50,M
1,4318,12 Angry Men (1957),1203,4,Drama,25,M
2,2756,Robocop 2 (1990),2986,2,Action|Crime|Sci-Fi,18,M
3,1706,Modern Times (1936),3462,5,Comedy,25,M
4,4813,Milk Money (1994),276,3,Comedy|Romance,35,F
...,...,...,...,...,...,...,...
800162,59,"Big Chill, The (1983)",2352,4,Comedy|Drama,50,F
800163,4458,So I Married an Axe Murderer (1993),543,4,Comedy|Romance|Thriller,25,F
800164,1234,Almost Famous (2000),3897,4,Comedy|Drama,18,M
800165,4864,"Fish Called Wanda, A (1988)",1079,5,Comedy,18,M


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler


df = complete_train
df.rename(columns={'sex': 'age', 'age': 'sex', 'release_date': 'genre', 'user_id': 'uid', 'movie_id': 'mid'}, inplace=True)
one_hot_genres = df['genre'].str.get_dummies(sep='|')
df = (df.drop(columns=['genre'])).join(one_hot_genres)
df['sex'] = df['sex'].replace({'M': 0, 'F': 1})
df['release_date'] = df['title'].str.extract(r'.*\((\d{4})\).*?$')
df['release_date'] = pd.to_numeric(df['release_date'], errors='coerce')
df['release_date_rank'] = df['release_date'].rank(method='average')
# Initialize a MinMaxScaler
scaler = MinMaxScaler()
# Rescale the ranked release dates to between 0 and 1
df['normalized_release_date'] = scaler.fit_transform(df['release_date_rank'].values.reshape(-1, 1))

unique_ages_sorted = sorted(df['age'].unique())

# Map each age to an equidistant point between 0 and 1
age_to_normalized = {age: i / (len(unique_ages_sorted) - 1) for i, age in enumerate(unique_ages_sorted)}

# Apply mapping to the original age values
df['normalized_age'] = df['age'].map(age_to_normalized)


df = df.drop(['title', 'age', 'release_date'	,'release_date_rank'], axis=1)
df

,uid,mid,rating,sex,Action,Adventure,Animation,Children's,Comedy,Crime,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,normalized_release_date,normalized_age
0,2592,1101,4,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0.341005,0.833333
1,4318,1203,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.061125,0.333333
2,2756,2986,2,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0.450953,0.166667
3,1706,3462,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.007844,0.333333
4,4813,276,3,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0.604616,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800162,59,2352,4,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.265322,0.833333
800163,4458,543,4,1,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0.554112,0.333333
800164,1234,3897,4,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1.000000,0.166667
800165,4864,1079,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.394491,0.166667


In [ ]:
def standardize_and_normalize(group):
    # Standardize
    mean_rating = group['rating'].mean()
    std_rating = group['rating'].std()
    if std_rating > 0:
        group['standardized_rating'] = (group['rating'] - mean_rating) / std_rating
    else:
        group['standardized_rating'] = 0.0  # Use float for consistency

    # Normalize
    min_std_rating = group['standardized_rating'].min()
    max_std_rating = group['standardized_rating'].max()
    range_std_rating = max_std_rating - min_std_rating
    if range_std_rating > 0:
        group['norm_std_rating'] = (group['standardized_rating'] - min_std_rating) / range_std_rating
    else:
        group['norm_std_rating'] = 0.5  # Use 0.5 to represent middle of the range if no variation

    return group

# Apply the function to each user's set of ratings
df = df.groupby('uid').apply(standardize_and_normalize).reset_index(drop=True)
df = df.drop(['standardized_rating', 'rating'], axis=1)
df.rename(columns={'normalized_release_date': 'release_year', 'norm_std_rating': 'rating', 'normalized_age': 'age'}, inplace=True)
df

<ipython-input-8-83bbf9da2eb3>:22: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('uid').apply(standardize_and_normalize).reset_index(drop=True)


,uid,mid,sex,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,release_year,age,rating
0,2592,1101,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0.341005,0.833333,0.750000
1,4318,1203,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.061125,0.333333,0.750000
2,2756,2986,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0.450953,0.166667,0.250000
3,1706,3462,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.007844,0.333333,1.000000
4,4813,276,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0.604616,0.500000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800162,59,2352,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.265322,0.833333,0.750000
800163,4458,543,1,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0.554112,0.333333,0.750000
800164,1234,3897,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1.000000,0.166667,0.666667
800165,4864,1079,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.394491,0.166667,1.000000


In [ ]:
df.columns

Index(['uid', 'mid', 'sex', 'Action', 'Adventure', 'Animation', 'Children's',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western', 'release_year', 'age', 'rating'],
      dtype='object')

In [ ]:
ready_df = df

# Baseline Recommender

In [ ]:
class TopPopRecommender():

    def fit(self, train):

        item_popularity = train[['movie_id','rating']].groupby(by='movie_id').count()

        self.train = train
        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = item_popularity.sort_values(by='rating',ascending=False).index


    def predict_top(self, user_id, at=5, remove_seen=True):

        if remove_seen:
            seen_items = self.train[self.train.user_id==user_id].movie_id.values
            unseen_items_mask = np.in1d(self.popular_items, seen_items, assume_unique=True, invert = True)
            unseen_items = self.popular_items[unseen_items_mask]
            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popular_items[0:at]

        return recommended_items

In [ ]:
#import csv


#topPopular = TopPopRecommender()
#topPopular.fit(complete_train)

# open the file in the write mode
#with open('/content/drive/My Drive/_Universität_HPI/Semester 6/Recommenders/Assignments/Assignment 1/Attempts/attemptsolutions/solution1.csv', 'w',encoding='UTF8') as f:
    # create the csv writer
#    writer = csv.writer(f)
    # write a row to the csv file
#    writer.writerow(['user_id', 'prediction'])
#    for user_id in test.user_id.unique():
#        relevant_items = topPopular.predict_top(user_id, at=25)
#        list_relevants = ' '.join([str(elem) for elem in relevant_items])
#        writer.writerow([str(user_id),list_relevants])

KeyError: "['movie_id'] not in index"

# FM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Flatten, Input, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Lambda, Add, Multiply
import tensorflow.keras.backend as K
from scipy import sparse


class HybridRecommenderSystem2M2():

    def __init__(self, num_users, num_movies, embedding_size=50):
        self.topPopular = TopPopRecommender()
        self.user_id_to_index = {}
        self.movie_id_to_index = {}
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.genre_columns = ['Action', 'Adventure', 'Animation', "Children's",
                              'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
                              'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
                              'Western']  # List of genres
        self.model = self._create_model()

    def _create_model(self):
        # Original inputs and embeddings
        user_input = Input(shape=(1,), name='user_input')
        user_embedding = Embedding(self.num_users, self.embedding_size, name='user_embedding')(user_input)
        user_vec = Flatten(name='user_flatten')(user_embedding)

        movie_input = Input(shape=(1,), name='movie_input')
        movie_embedding = Embedding(self.num_movies, self.embedding_size, name='movie_embedding')(movie_input)
        movie_vec = Flatten(name='movie_flatten')(movie_embedding)

        genres_input = Input(shape=(len(self.genre_columns),), name='genres_input')
        sex_input = Input(shape=(1,), name='sex_input')
        age_input = Input(shape=(1,), name='age_input')

        concatenated = Concatenate()([user_vec, movie_vec, genres_input, sex_input, age_input])

        # Factorization Machine-like layer for interactions
        summed_features_emb = Lambda(lambda x: K.sum(x, axis=1, keepdims=True))(concatenated)  # Keep dimension
        summed_square = Multiply()([summed_features_emb, summed_features_emb])
        squared_features_emb = Multiply()([concatenated, concatenated])
        squared_sum = Lambda(lambda x: K.sum(x, axis=1, keepdims=True))(squared_features_emb)  # Keep dimension
        sub = Lambda(lambda x: x[0] - x[1])([summed_square, squared_sum])
        fm_layer = Lambda(lambda x: x * 0.5)(sub)


        # Combine FM layer with deep layers
        dense = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(fm_layer)
        dense = Dropout(0.5)(dense)
        dense = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(dense)
        dense = Dropout(0.5)(dense)
        output = Dense(1, activation='sigmoid')(dense)

        model = Model(inputs=[user_input, movie_input, genres_input, sex_input, age_input], outputs=output)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mse'])  # Use MSE for both loss and metric
        #model.compile(keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])
        #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model



    def fit(self, df, pop_source):
        self.topPopular.fit(pop_source)
        self.df = df  # Store the original DataFrame for later use
        # Create mappings for user and movie IDs to zero-based indices
        unique_user_ids = df['uid'].unique()
        self.user_id_to_index = {uid: index for index, uid in enumerate(unique_user_ids)}

        unique_movie_ids = df['mid'].unique()
        self.movie_id_to_index = {mid: index for index, mid in enumerate(unique_movie_ids)}



        # Also, store the inverse mapping for movies to facilitate lookup in recommend_movies
        self.index_to_movie_id = {index: mid for mid, index in self.movie_id_to_index.items()}

        # Update num_users and num_movies based on actual unique IDs
        #self.num_users = len(unique_user_ids)
        #self.num_movies = len(unique_movie_ids)

        # Initialize the sparse matrix with new dimensions
        rating_matrix = sparse.lil_matrix((self.num_users, self.num_movies))

        # Populate the matrix with known ratings, using the mapping
        for index, row in df.iterrows():
            user_idx = self.user_id_to_index[row['uid']]
            movie_idx = self.movie_id_to_index[row['mid']]
            rating_matrix[user_idx, movie_idx] = row['rating']

        # Convert matrix to csr format
        rating_matrix = rating_matrix.tocsr()

        # Create expanded arrays
        expanded_user_ids = np.repeat(range(self.num_users), self.num_movies)
        expanded_movie_ids = np.tile(range(self.num_movies), self.num_users)
        expanded_ratings = rating_matrix[expanded_user_ids, expanded_movie_ids].A1  # Flatten to 1D array

        # Prepare genre, sex, and age information
        expanded_genres = np.zeros((self.num_users * self.num_movies, len(self.genre_columns)))
        expanded_sex = np.zeros(self.num_users * self.num_movies)
        expanded_age = np.zeros(self.num_users * self.num_movies)

        for uid, idx in self.user_id_to_index.items():
            user_rows = df[df['uid'] == uid]
            if not user_rows.empty:
                user_info = user_rows.iloc[0]
                expanded_sex[idx::self.num_users] = user_info['sex']
                expanded_age[idx::self.num_users] = user_info['age']

        for mid, idx in self.movie_id_to_index.items():
            movie_rows = df[df['mid'] == mid]
            if not movie_rows.empty:
                expanded_genres[idx * self.num_users:(idx + 1) * self.num_users] = movie_rows[self.genre_columns].values[0]

        # Fit the model with expanded data
        self.model.fit([expanded_user_ids, expanded_movie_ids, expanded_genres, expanded_sex, expanded_age],
                      expanded_ratings, epochs=4, batch_size=256, validation_split=0.2) # 64





    def evaluate_model(self, val_df):
        user_ids = val_df['uid'].values
        movie_ids = val_df['mid'].values
        genres = val_df[self.genre_columns].values
        sex = val_df['sex'].values
        age = val_df['age'].values
        true_ratings = val_df['rating'].values

        predictions = self.model.predict([user_ids, movie_ids, genres, sex, age])
        mse = mean_squared_error(true_ratings, predictions)
        print(f'Mean Squared Error on Validation Set: {mse}')

    def recommend_movies(self, user_id, top_n=25, batch_size=1024):
        # Check if the user_id and movie_ids are in the mapping
        if user_id not in self.user_id_to_index:
            return []  # User not found

        user_idx = self.user_id_to_index[user_id]

        # Filter for unseen movies
        seen_movie_ids = self.df[self.df['uid'] == user_id]['mid'].unique()
        unseen_movie_ids = self.topPopular.predict_top(user_id, at=150) #[mid for mid in self.df['mid'].unique() if mid not in seen_movie_ids]
        unseen_movie_indices = [self.movie_id_to_index[mid] for mid in unseen_movie_ids if mid in self.movie_id_to_index]

        # Prepare arrays for batch prediction
        num_unseen_movies = len(unseen_movie_indices)
        batch_user_ids = np.full(shape=(num_unseen_movies,), fill_value=user_idx)
        batch_movie_ids = np.array(unseen_movie_indices)

        # Prepare user-specific features
        user_info = self.df[self.df['uid'] == user_id].iloc[0] if not self.df[self.df['uid'] == user_id].empty else None
        batch_sex = np.full(shape=(num_unseen_movies,), fill_value=user_info['sex'] if user_info is not None else 0)
        batch_age = np.full(shape=(num_unseen_movies,), fill_value=user_info['age'] if user_info is not None else 0)

        # Preparing genres data for unseen movies
        batch_genres = np.zeros((num_unseen_movies, len(self.genre_columns)))
        for i, movie_index in enumerate(unseen_movie_indices):
            if movie_index in self.index_to_movie_id:  # Ensure movie index is valid
                mid = self.index_to_movie_id[movie_index]
                if not self.df[self.df['mid'] == mid].empty:
                    batch_genres[i] = self.df[self.df['mid'] == mid].iloc[0][self.genre_columns].values

        # Predict ratings in batches to avoid memory issues
        predicted_ratings = []
        for i in range(0, num_unseen_movies, batch_size):
            batch_preds = self.model.predict(
                [batch_user_ids[i:i + batch_size],
                batch_movie_ids[i:i + batch_size],
                batch_genres[i:i + batch_size],
                batch_sex[i:i + batch_size],
                batch_age[i:i + batch_size]]
            ).flatten()
            predicted_ratings.extend(batch_preds)

        # Get top N movie recommendations
        predicted_ratings = np.array(predicted_ratings)
        top_indices = np.argsort(predicted_ratings)[-top_n:][::-1]  # Indices of top-n ratings
        recommended_movie_indices = batch_movie_ids[top_indices]
        recommended_movie_ids = [self.index_to_movie_id[idx] for idx in recommended_movie_indices if idx in self.index_to_movie_id]

        return recommended_movie_ids







In [ ]:
from sklearn.model_selection import train_test_split

num_users = ready_df['uid'].nunique()
num_movies = ready_df['mid'].nunique()

# Initialize the recommender system
recommender2M2 = HybridRecommenderSystem2M2(num_users, num_movies)

# Assuming 'df' is your DataFrame
train_df, test_df = train_test_split(ready_df, test_size=0.2)  # Split 20% of the data as the test set

recommender2M2.fit(train_df)
recommender2M2.evaluate_model(test_df)
# Get recommendations for a specific user
user_id = 1  # Change this to the ID of the user for whom you want recommendations
recommended_movies = recommender2M2.recommend_movies(user_id, top_n=25)

print(f"Top 25 recommended movies for user {user_id}: {recommended_movies}")

Epoch 1/2
  5209/275349 [..............................] - ETA: 1:16:44 - loss: 0.1080 - accuracy: 0.9683

KeyboardInterrupt: 

Memoryoptimised:
  - 0.447 (one epoch = 20mins)

In [ ]:
import csv
from tqdm import tqdm


num_users = ready_df['uid'].nunique()
num_movies = ready_df['mid'].nunique()

# Initialize the recommender system
recommender2M2 = HybridRecommenderSystem2M2(num_users, num_movies)
recommender2M2.fit(ready_df, complete_train_og)

with open('/content/drive/My Drive/RecommendersAssignment/FMAhhhhh.csv', 'w',encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    # write a row to the csv file
    writer.writerow(['user_id', 'prediction'])
    for user_id in tqdm(test.user_id.unique()):
        #n_ratings = ready_df[ready_df['uid']==user_id].movie_id.count()
        #if(n_ratings > 30):
        relevant_items = recommender2M2.recommend_movies(user_id, top_n=25)
        #else:
        # relevant_items = topPopular.predict_top(user_id, at=25)
        list_relevants = ' '.join([str(elem) for elem in relevant_items])
        writer.writerow([str(user_id),list_relevants])

  0%|          | 0/6037 [00:00<?, ?it/s]

114/114 [==============================] - 0s 2ms/step


  0%|          | 1/6037 [00:08<14:16:06,  8.51s/it]

114/114 [==============================] - 0s 2ms/step


  0%|          | 2/6037 [00:14<11:24:08,  6.80s/it]

...

  0%|          | 28/6037 [02:47<9:38:10,  5.77s/it]

111/111 [==============================] - 0s 2ms/step


  0%|          | 29/6037 [02:53<9:55:45,  5.95s/it]

112/112 [==============================] - 0s 2ms/step


  0%|          | 30/6037 [02:59<9:44:23,  5.84s/it]

Ansatz für 2:
  - Also das beste Factorization Machine oder vielleicht Deep Learning benutzen
  - Als erstes versuchen die Bewertungen für jede Person einzeln zu normalisieren (danach gucken, ob es besser ist, mit binär und für die besten 30% 1 oder so)
  - Alle Datenspalten aus der Datenbank hinzunehen
  - Dann einfach rawdog probieren und danach iwie die populärsten bevorzugen oder so
  - Noch weitere spalten: Die genres in One-Hot-Encoding
  - Wenn ich übertreiben will, dann irgendwelches Assembly Modell Shit machen oder so
  - Mit first und second order und so herumprobieren
  - Definitiv auch die release Daten als Faktor einbeziehen, vielleicht die release Jahre normalisieren (kann glaube ich als faktor für entscheidungen sehr stark und wichtig sein!!
  - Vielleicht iwie, den rating wert (0 bis 1) multiplizieren mit dem ranking wert des Filmes (mit Toprecommender alle ungesehen Filme sortieren und populärsten mit 1 starten lassen und unpopulärsten 0.1 enden lassen und dann den rating mit dem ranking wert multiplizieren für finales ergebnis oder so :))
  